In [ ]:
import pandas as pd
from tqdm import tqdm
from utils.evaluator import evaluate
dataset = 'Apache'
output_file = f'outputs/parser/LogBatcher_0shot_32candidate_10batchsize_2000chunksize_full/{dataset}_2k.log_structured.csv'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
# a =evaluate(output_file=output_file, groundtruth_file=groundtruth_file,dataset=dataset)
# print(a)
df1 = pd.read_csv(output_file)
logs1 = df1['EventTemplate'].tolist()
df2 = pd.read_csv(groundtruth_file)
logs2 = df2['EventTemplate'].tolist()

count = 0
length = len(logs1)
for i in tqdm(range(length)):
    if logs1[i] == logs2[i]:
        count += 1
print(count / length)
print(count)


In [ ]:
from utils.postprocess import correct_single_template

print(correct_single_template('<*> "<*> <*> 1.2.3.4:<*> status: <*> len: <*> time: <*>'))

In [ ]:
import re
import pandas as pd
dataset = 'HDFS'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
df = pd.read_csv(groundtruth_file)
# 定义更新规则的函数
def update_content(content):
    # 使用正则表达式匹配并更新内容
    pattern = r'(BLOCK\* NameSystem\.allocateBlock: [^ ]*) ([^ ]*)'
    replacement = r'\1. \2'
    return re.sub(pattern, replacement, content)

# 更新Content列
df['Content'] = df['Content'].apply(update_content)

# 保存修改后的CSV文件
df.to_csv('updated_file.csv', index=False)

In [ ]:
import pandas as pd
from tqdm import tqdm
from utils.evaluator import evaluate
dataset = 'BGL'
output_file = f'outputs/parser/LILAC_full/{dataset}_full.log_structured.csv'
# output_file ='outputs/parser/LogBatcher_0shot_32candidate_10batchsize/BGL_2k.log_structured.csv'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
# groundtruth_file = 'dataset/BGL/BGL_2k.log_structured_corrected.csv'
a =evaluate(output_file=output_file, groundtruth_file=groundtruth_file,dataset=dataset)
print(a)




In [2]:
cost  =[[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "proxy.cse.cuhk.edu.hk:5070 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS\n182.254.114.110:80 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS\ngithub-windows.s3.amazonaws.com:443 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS\ntc.dl.pinyin.sogoucdn.com:80 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS\n203.205.146.15:80 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS\npeople-pa.clients6.google.com:443 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS\nf-log-extension.grammarly.io:443 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS\nxf-com-update-doctor.qq.com:443 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS\nsrc.wei.focus.cn:80 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS\nvideo-hkg3-2.xx.fbcdn.net:443 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "r2---sn-i3b7kn7r.googlevideo.com:443 close, 2883 bytes (2.81 KB) sent, 90785 bytes (88.6 KB) received, lifetime 00:30\nproxy.cse.cuhk.edu.hk:5070 close, 3425 bytes (3.34 KB) sent, 212164 bytes (207 KB) received, lifetime 00:18\nf-log-extension.grammarly.io:443 close, 1363 bytes (1.33 KB) sent, 3549 bytes (3.46 KB) received, lifetime 00:33\n183.62.156.108:22 close, 89652 bytes (87.5 KB) sent, 599249 bytes (585 KB) received, lifetime 37:51\nlong.open.weixin.qq.com:443 close, 1985 bytes (1.93 KB) sent, 4832 bytes (4.71 KB) received, lifetime 00:02\nbolt.dropbox.com:443 close, 15427 bytes (15.0 KB) sent, 8268 bytes (8.07 KB) received, lifetime 15:31\ntc.dl.pinyin.sogoucdn.com:80 close, 1537 bytes (1.50 KB) sent, 7856 bytes (7.67 KB) received, lifetime 01:00\nscontent-hkg3-2.xx.fbcdn.net:443 close, 3732 bytes (3.64 KB) sent, 804155 bytes (785 KB) received, lifetime 01:10\nassets-cdn.github.com:443 close, 1227 bytes (1.19 KB) sent, 4890 bytes (4.77 KB) received, lifetime 06:19\nc.csdnimg.cn:80 close, 1314 bytes (1.28 KB) sent, 27289 bytes (26.6 KB) received, lifetime 03:09"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "proxy.cse.cuhk.edu.hk:5070 close, 0 bytes sent, 0 bytes received, lifetime 00:17\ntxt.go.sohu.com:80 close, 606 bytes sent, 857 bytes received, lifetime 00:07\nf-log-extension.grammarly.io:443 close, 0 bytes sent, 0 bytes received, lifetime 00:11\nblog.csdn.net:80 close, 670 bytes sent, 421 bytes received, lifetime 00:29\ncidian.youdao.com:80 close, 923 bytes sent, 456 bytes received, lifetime 09:53\nr6---sn-i3b7knl6.googlevideo.com:443 close, 733 bytes sent, 151 bytes received, lifetime 00:15\njs.passport.qihucdn.com:80 close, 496 bytes sent, 426 bytes received, lifetime 00:16\n24.135.95.248:443 close, 131 bytes sent, 153 bytes received, lifetime 00:12\ntc.dl.pinyin.sogoucdn.com:80 close, 799 bytes sent, 346 bytes received, lifetime 01:00\ngtms03.alicdn.com:80 close, 399 bytes sent, 498 bytes received, lifetime 02:06"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "proxy.cse.cuhk.edu.hk:5070 close, 1143 bytes (1.11 KB) sent, 365 bytes received, lifetime 00:01\nuser-images.githubusercontent.com:443 close, 335 bytes sent, 4369 bytes (4.26 KB) received, lifetime 03:15\np2.qhimg.com:80 close, 230 bytes sent, 1125 bytes (1.09 KB) received, lifetime 00:09\ndslb.cdn.krcom.cn:80 close, 446 bytes sent, 195160 bytes (190 KB) received, lifetime 00:59\nqa.sockets.stackexchange.com:443 close, 1603 bytes (1.56 KB) sent, 527 bytes received, lifetime 22:32\n13.client-channel.google.com:443 close, 474 bytes sent, 4464 bytes (4.35 KB) received, lifetime 01:30\nf-log-extension.grammarly.io:443 close, 643 bytes sent, 3276 bytes (3.19 KB) received, lifetime 00:19\nlong.open.weixin.qq.com:443 close, 1910 bytes (1.86 KB) sent, 276 bytes received, lifetime 00:28\nstatic.doubleclick.net:443 close, 2553 bytes (2.49 KB) sent, 641 bytes received, lifetime 06:39\nzhihu-web-analytics.zhihu.com:443 close, 343 bytes sent, 3122 bytes (3.04 KB) received, lifetime 00:17"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "proxy.cse.cuhk.edu.hk:5070 close, 403 bytes sent, 426 bytes received, lifetime <1 sec\ncnzz.mmstat.com:80 close, 686 bytes sent, 579 bytes received, lifetime <1 sec\nclients6.google.com:443 close, 568 bytes sent, 156 bytes received, lifetime <1 sec\nlog.getdropbox.com:80 close, 739 bytes sent, 404 bytes received, lifetime <1 sec\nfs-conn-doctor.qq.com:443 close, 184 bytes sent, 134 bytes received, lifetime <1 sec\ncgi.qqweb.qq.com:80 close, 477 bytes sent, 448 bytes received, lifetime <1 sec\n203.205.151.204:80 close, 357 bytes sent, 484 bytes received, lifetime <1 sec\np3p.sogou.com:80 close, 441 bytes sent, 139 bytes received, lifetime <1 sec\ntcpconn3.tencent.com:443 close, 149 bytes sent, 121 bytes received, lifetime <1 sec\nyt3.ggpht.com:443 close, 733 bytes sent, 229 bytes received, lifetime <1 sec"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "2052.flash2-http.qq.com:80 close, 466 bytes sent, 125682 bytes (122 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1165 bytes (1.13 KB) sent, 815 bytes received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 704 bytes sent, 2476 bytes (2.41 KB) received, lifetime <1 sec\nc.baidu.com:80 close, 1052 bytes (1.02 KB) sent, 113 bytes received, lifetime <1 sec\nupdate.googleapis.com:443 close, 298 bytes sent, 4250 bytes (4.15 KB) received, lifetime <1 sec\n12.client-channel.google.com:443 close, 474 bytes sent, 4042 bytes (3.94 KB) received, lifetime <1 sec\nd0.sina.com.cn:443 close, 328 bytes sent, 3654 bytes (3.56 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1301 bytes (1.27 KB) sent, 434 bytes received, lifetime <1 sec\npuui.qpic.cn:80 close, 379 bytes sent, 7852 bytes (7.66 KB) received, lifetime <1 sec\ntc.dl.pinyin.sogoucdn.com:80 close, 158 bytes sent, 46624 bytes (45.5 KB) received, lifetime <1 sec"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "tcpconn6.tencent.com:443 error : A connection request was canceled before the completion.\noimagec7.ydstatic.com:80 error : A connection request was canceled before the completion.\noimagea5.ydstatic.com:80 error : A connection request was canceled before the completion.\noimageb3.ydstatic.com:80 error : A connection request was canceled before the completion.\noimageb1.ydstatic.com:80 error : A connection request was canceled before the completion.\noimagea3.ydstatic.com:80 error : A connection request was canceled before the completion.\noimagec1.ydstatic.com:80 error : A connection request was canceled before the completion.\nhm.baidu.com:80 error : A connection request was canceled before the completion.\ncsdnimg.cn:80 error : A connection request was canceled before the completion.\nclient-cf.dropbox.com:443 error : A connection request was canceled before the completion."
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "qa.sockets.stackexchange.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061\n13.client-channel.google.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061\nwww.google.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061\nclient-cf.dropbox.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061\n16.client-channel.google.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061\nbolt.dropbox.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061\nmtalk.google.com:5228 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061\nclient-lb.dropbox.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "proxy.cse.cuhk.edu.hk:5070 close, 1268 bytes (1.23 KB) sent, 6274 bytes (6.12 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1327 bytes (1.29 KB) sent, 3250 bytes (3.17 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1107 bytes (1.08 KB) sent, 29322 bytes (28.6 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1133 bytes (1.10 KB) sent, 11393 bytes (11.1 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1119 bytes (1.09 KB) sent, 3210 bytes (3.13 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1372 bytes (1.33 KB) sent, 1224 bytes (1.19 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1986 bytes (1.93 KB) sent, 1180 bytes (1.15 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1448 bytes (1.41 KB) sent, 1794 bytes (1.75 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1080 bytes (1.05 KB) sent, 4111 bytes (4.01 KB) received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 5349 bytes (5.22 KB) sent, 1767 bytes (1.72 KB) received, lifetime <1 sec"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "223.167.104.147:80 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001\nmtalk.google.com:5228 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001\ns1.bdstatic.com:80 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001\nss.bdimg.com:80 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001\nbolt.dropbox.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001\nwww.qq.com:80 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001\n203.205.151.204:80 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001\nget.sogou.com:80 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001\n120.204.0.139:8080 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001\n101.226.211.46:8080 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "formi.baidu.com:843 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403\n127.0.0.1:135 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403\n203.205.129.102:8080 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403\nmtalk.google.com:5228 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403\nupdate.hicloud.com:8180 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403\nlocal-p2p.qq.com:443 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 503\nqd-update.qq.com:8080 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403\ni2.itc.cn:80 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 503"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "proxy.cse.cuhk.edu.hk:5070 close, 2933 bytes (2.86 KB) sent, 11721005 bytes (11.1 MB) received, lifetime 02:48\nproxy.cse.cuhk.edu.hk:5070 close, 47856 bytes (46.7 KB) sent, 4090387 bytes (3.90 MB) received, lifetime 01:01\nproxy.cse.cuhk.edu.hk:5070 close, 31257 bytes (30.5 KB) sent, 1846301 bytes (1.76 MB) received, lifetime 01:33\nproxy.cse.cuhk.edu.hk:5070 close, 7437 bytes (7.26 KB) sent, 2235596 bytes (2.13 MB) received, lifetime 00:07\nr6---sn-i3b7kn7d.googlevideo.com:443 close, 12789 bytes (12.4 KB) sent, 13833013 bytes (13.1 MB) received, lifetime 01:01\nr6---sn-i3b7knez.googlevideo.com:443 close, 13118 bytes (12.8 KB) sent, 3242984 bytes (3.09 MB) received, lifetime 00:31\nr2---sn-i3b7kne6.googlevideo.com:443 close, 17742 bytes (17.3 KB) sent, 11581393 bytes (11.0 MB) received, lifetime 01:08\nvideo-hkg3-2.xx.fbcdn.net:443 close, 58373 bytes (57.0 KB) sent, 8896991 bytes (8.48 MB) received, lifetime 02:25\nr1---sn-i3belnez.googlevideo.com:443 close, 10046 bytes (9.81 KB) sent, 5657896 bytes (5.39 MB) received, lifetime 00:54"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "tcpconn4.tencent.com:80 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.\n86.99.222.235:443 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.\nconfig.pinyin.sogou.com:80 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly."
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "183.62.156.108:22 open through proxy socks.cse.cuhk.edu.hk:5070 SOCKS5"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "proxy.cse.cuhk.edu.hk:5070 close, 1143 bytes (1.11 KB) sent, 365 bytes received, lifetime 00:01\ncontent.googleapis.com:443 close, 2094 bytes (2.04 KB) sent, 623 bytes received, lifetime 04:00\nrs.sinajs.cn:80 close, 1187 bytes (1.15 KB) sent, 472 bytes received, lifetime 00:12\nwww.weibo.com:80 close, 11058 bytes (10.7 KB) sent, 451 bytes received, lifetime 00:07\nplay.google.com:443 close, 2130 bytes (2.08 KB) sent, 1009 bytes received, lifetime 03:24\nproxy.cse.cuhk.edu.hk:5070 close, 1731 bytes (1.69 KB) sent, 864 bytes received, lifetime 02:53\nqa.sockets.stackexchange.com:443 close, 1603 bytes (1.56 KB) sent, 527 bytes received, lifetime 22:32\nstatic.doubleclick.net:443 close, 2553 bytes (2.49 KB) sent, 641 bytes received, lifetime 06:39\ndj1.baidu.com:80 close, 1369 bytes (1.33 KB) sent, 289 bytes received, lifetime 00:30\nlong.open.weixin.qq.com:443 close, 1910 bytes (1.86 KB) sent, 276 bytes received, lifetime 00:27"
    }
],
[
    {
        "role": "system",
        "content": "You will be provided with some log messages separated by line break. You must abstract variables with `{{placeholders}}` to extract the corresponding template. There might be no variables in the log message.\nPrint the input log's template delimited by backticks."
    },
    {
        "role": "user",
        "content": "proxy.cse.cuhk.edu.hk:5070 close, 1118 bytes (1.09 KB) sent, 340 bytes received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1301 bytes (1.27 KB) sent, 434 bytes received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1180 bytes (1.15 KB) sent, 337 bytes received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 2035 bytes (1.98 KB) sent, 469 bytes received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1920 bytes (1.87 KB) sent, 338 bytes received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1808 bytes (1.76 KB) sent, 346 bytes received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 1104 bytes (1.07 KB) sent, 406 bytes received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 2587 bytes (2.52 KB) sent, 317 bytes received, lifetime <1 sec\nproxy.cse.cuhk.edu.hk:5070 close, 2844 bytes (2.77 KB) sent, 279 bytes received, lifetime <1 sec\nc.baidu.com:80 close, 1052 bytes (1.02 KB) sent, 113 bytes received, lifetime <1 sec"
    }
]
]

LogBatcher_0shot_32candidate_10batchsize_2000chunksize_full_time

In [3]:
import json

import tiktoken
def count_message_tokens(messages, model_name):
    # 根据模型名称加载合适的编码器
    if model_name == "gpt-4":
        encoder = tiktoken.encoding_for_model("gpt-4")
    elif model_name == "gpt-3.5-turbo":
        encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
    else:
        raise ValueError("未知的模型名称")

    # 初始化token计数
    token_count = 0

    # 计算每个消息的token数
    for message in messages:
        role_tokens = encoder.encode(message['role'])
        content_tokens = encoder.encode(message['content'])
        token_count += len(role_tokens) + \
            len(content_tokens) + 4  # 加上特殊的消息分隔符的token数

    return token_count

dataset = 'BGL'

counts_token = 0
counts_message = 0

# # 存储解析后的日志列表
# message_list = []
# # load every message
# with open('outputs/cost/cost_full_LILAC_32_3_BGL.json', 'r') as file:
#     for line in file:
#         if line.strip() == '[':
#             list_str = ''
#             start_load = True
#         if line.strip() == ']':
#             list_str += line
#             message = json.loads(list_str)
#             message_list.append(message)
#             start_load = False
#         if start_load:
#             list_str += line
# print(len(message_list))

# remove the same log messages
# def make_hashable(log_list):
#     return tuple(tuple(sorted(d.items())) for d in log_list)
# message_list = list(set(make_hashable(log_list) for log_list in message_list))
# message_list = [list(map(dict, log_list)) for log_list in message_list]
# print(len(message_list))

for message in cost:
    counts_token += count_message_tokens(message, 'gpt-3.5-turbo')
    counts_message += 1
print(counts_token, counts_token/counts_message)


5741 358.8125


In [6]:
if -1:
    print(1)

1


In [ ]:
import string
import re

def clean(s):
    """ Preprocess log message
    Parameters
    ----------
    s: str, raw log message
    Returns
    -------
    str, preprocessed log message without number tokens and special characters
    """
    s = re.sub(':|\(|\)|=|,|"|\{|\}|@|$|\[|\]|\||;|\.', ' ', s)
    s = " ".join([word.lower() if word.isupper() else word for word in s.strip().split()])
    s = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', s))
    s = " ".join([word for word in s.split() if not bool(re.search(r'\d', word))])
    trantab = str.maketrans(dict.fromkeys(list(string.punctuation)))
    s = s.translate(trantab)
    s = " ".join([word.lower().strip() for word in s.strip().split()])
    return s

print(clean('authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root'))
print(clean('10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2580249'))

In [ ]:
import re
p_tokens = ['HTTP/1.<*>']
new_p_tokens = []
for p_token in p_tokens:
    if re.match(r'^(\/[^\/]+)+$', p_token):
        p_token = '<*>'
    new_p_tokens.append(p_token)
print(new_p_tokens)

In [ ]:
import re
line = 'iar 00106448 dear 0246dacc'

Rexs =  [
            "((?<=[^A-Za-z0-9])|^)(([0-9a-f]{2,}:){3,}([0-9a-f]{2,}))((?=[^A-Za-z0-9])|$)",
            "((?<=[^A-Za-z0-9])|^)(\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3})((?=[^A-Za-z0-9])|$)",
            "((?<=[^A-Za-z0-9])|^)([0-9a-f]{6,} ?){3,}((?=[^A-Za-z0-9])|$)",
            "((?<=[^A-Za-z0-9])|^)([0-9A-F]{4} ?){4,}((?=[^A-Za-z0-9])|$)",
            "((?<=[^A-Za-z0-9])|^)(0x[a-f0-9A-F]+)((?=[^A-Za-z0-9])|$)",
            "((?<=[^A-Za-z0-9])|^)([\\-\\+]?\\d+)((?=[^A-Za-z0-9])|$)",
            "(?<=executed cmd )(\".+?\")",
            "(\\w+):\\/\\/([^/:]+)(:\\d*)?([^# ]*)",
            "\\/[<\\*>|\\w]+\\/([<\\*>|\\w]+\\/?)+",
            "(<\\*>)(\\s*<\\*>){4,}",
            "(<\\*>)(##*<\\*>){2,}",
            "(\\w{0,1}<\\*>-){2,}(\\w|<\\*>)*",
            "<\\*>(\\.\\w+|\\(\\)|\\/)",
            "(false|true|root|ROOT)"
        ]
for rex in Rexs:
    line = re.sub(rex, "", line)
print(line)

In [9]:
import pandas as pd
dataset = 'proxifier'
df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected__.csv')

# def modify_content(row):
#     content = row['Content']
#     if 'lifetime ' in content:
#         # if 'KB) received' in content and not 'KB) sent' in content and not 'MB' in content and not '<' in content:
#         #     print('chage')
#         #     return '<*> close, <*> bytes sent, <*> bytes (<*> KB) received, lifetime <*>'
        
#         if not 'KB' in content and not '<' in content:
#             return '<*> close, <*> bytes sent, <*> bytes received, lifetime <*>'
#         if 'KB) sent' in content and not 'KB) received' in content and '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes (<*> KB) sent, <*> bytes received, lifetime <1 sec'
#         if not 'KB) sent' in content and 'KB) received' in content and '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes sent, <*> bytes (<*> KB) received, lifetime <1 sec'
#         if 'KB) sent' in content and not 'KB) received' in content and not '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes (<*> KB) sent, <*> bytes received, lifetime <*>'
#         if not 'KB) sent' in content and 'KB) received' in content and not '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes sent, <*> bytes (<*> KB) received, lifetime <*>'
#         if 'KB) sent' in content and 'KB) received' in content and not '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> KB) received, lifetime <*>'
#         if 'KB) sent' in content and 'KB) received' in content and '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> KB) received, lifetime <1 sec'
#         if 'KB) sent' in content and 'MB) received' in content and not '<' in content:
#             return '<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> MB) received, lifetime <*>'
#         else:
#             return '<*> close, <*> bytes sent, <*> bytes received, lifetime <1 sec'
#     else:
#         return row['EventTemplate']
def modify_content(template):
    return template.replace('<1 sec', '<*>')
df['EventTemplate'] = df['EventTemplate'].apply(modify_content)
df.to_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv', index=False)

In [ ]:
import re
import tiktoken
import json



def count_prompt_tokens(prompt, model_name):
    """
    Count the number of tokens in the prompt
    Models supported: gpt-4, gpt-3.5-turbo
    """
    if model_name == "gpt-4":
        encoder = tiktoken.encoding_for_model("gpt-4")
    elif model_name == "gpt-3.5-turbo":
        encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
    else:
        raise ValueError("未知的模型名称")

    # 计算编码后的token数
    prompt_tokens = encoder.encode(prompt)
    return len(prompt_tokens)

with open('outputs/cost/LogBatcher_0shot_32candidate_10batchsize_testtesttest.json', 'r') as file:
    prompts = json.load(file)

count = 0
for prompt in prompts:
    count += count_prompt_tokens(prompt, 'gpt-3.5-turbo') 
print(count)    
print(count/len(prompts))
print(len(prompts))

In [10]:
from utils.matching import extract_variables
from utils.cluster import tokenize
import re
log1 = 'proxy.cse.cuhk.edu.hk:5070 close, 1143 bytes (1.11 KB) sent, 365 bytes received, lifetime 00:01'
log2 = 'authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 user=<*>'
template = 'authentication failure; logname= uid=<*> euid=<*> tty=NODEVssh ruser= rhost=<*>'


groups = extract_variables(log2, template)
print(groups)
if groups:
    for group in groups:
        if len(tokenize(group)) >= 2:
            print('no')
        else:
            print(tokenize(group))

('0', '0', '218.188.2.4 user=<*>')
['0']
['0']
['user']
